In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time as ttime_

In [2]:
chr_options = Options()
chr_options.add_argument("--start-maximized")
s = Service(r'C:\jupiter\ii\chromedriver.exe')
driver = webdriver.Chrome(service=s, options=chr_options)

In [3]:
link = f'https://www.flashscore.com.ua/'
driver.get(link)

In [4]:
dict_ = {'ИЗБРАННОЕ':0,
'ФУТБОЛ':1,
'ХОККЕЙ':2,
'ТЕННИС':3,
'БАСКЕТБОЛ':4,
'ВОЛЕЙБОЛ':5,
'ГАНДБОЛ':6}

In [5]:
sport = 'ХОККЕЙ'
param = ['Броски в створ ворот','Удаления']

In [6]:
def create_df(sport,param):
    driver.find_elements(By.CLASS_NAME, "menuTop__text")[dict_[sport]].click()
    match = driver.find_elements(By.CLASS_NAME, 'event__match.event__match--twoLine')
    match_id = []
    for i in match:
        match_id.append(i.get_attribute('id'))
    sp_id = []
    for i in match_id:
        sp_id.append(i[4:])
    final = []
    for id_ in sp_id:
        url = 'https://www.flashscore.com.ua/match/'+id_+'/#/match-summary/match-summary'
        driver.get(url)
        home_team = driver.find_elements(By.CLASS_NAME, 'duelParticipant__home')[0].text
        away_team = driver.find_elements(By.CLASS_NAME, 'duelParticipant__away')[0].text
        rezalt = driver.find_elements(By.CLASS_NAME, 'detailScore__wrapper')[0].text.splitlines()
        rezalt.remove('-')
        status = driver.find_elements(By.CLASS_NAME, 'detailScore__status')[0].text
        time = driver.find_elements(By.CLASS_NAME, 'duelParticipant__startTime')[0].text
        liga = driver.find_elements(By.CLASS_NAME, 'tournamentHeader__country')[0].text
        url_stat = 'https://www.flashscore.com.ua/match/'+id_+'/#/match-summary/match-statistics/0'
        driver.get(url_stat)
        ttime_.sleep(1)
        sp = driver.find_elements(By.CLASS_NAME, 'stat__category')
        stat_m = []
        for i in sp:
            stat_m+=[i.text.splitlines()]
        dict_info = {}
        for i in stat_m:
            dict_info[i[1]] = [i[0],i[2]]
        if len(rezalt)==2:
            all_info = [liga,time,status,home_team,away_team,rezalt[0],rezalt[1]]
        else:
            rezalt = ['-','-']
        all_info = [liga,time,status,home_team,away_team,rezalt[0],rezalt[1]]
        sp_inform = ['Liga','time','status','Home_Team','Away_Team','rezalt_home','rezalt_away']
        main_info = pd.DataFrame(all_info,index = sp_inform).T
        main_info = main_info.reset_index(drop = True)
        dop_info = pd.DataFrame(dict_info)
        if len(dop_info)>0:
            dop_info = dop_info[param]
        else:
            dop_info = pd.DataFrame(['_' for _ in range(len(param))],index = param).T
            dop_info.loc[1,:] = dop_info.loc[0,:]
        home_info = dop_info.loc[[0],:]
        home_info = home_info.add_suffix('_home')
        away_info = dop_info.loc[[1],:]
        away_info = away_info.add_suffix('_away').reset_index(drop = True)
        final_dop_info = pd.merge(home_info,away_info,left_index = True, right_index = True)
        final += [pd.merge(main_info,final_dop_info,left_index = True, right_index = True)]
    return final

In [7]:
final = create_df(sport,param)
final = pd.concat(final)

In [8]:
final

,Liga,time,status,Home_Team,Away_Team,rezalt_home,rezalt_away,Броски в створ ворот_home,Удаления_home,Броски в створ ворот_away,Удаления_away
0,ЕВРОПА: ЛИГА ЧЕМПИОНОВ - ПЛЕЙ-ОФФ - 1/2,17.01.2023 20:05,,Лулео,Фрелунда,-,-,_,_,_,_
0,ЕВРОПА: ЛИГА ЧЕМПИОНОВ - ПЛЕЙ-ОФФ - 1/2,17.01.2023 22:15,,Цуг,Таппара,-,-,_,_,_,_
0,США: НХЛ,17.01.2023 00:00,ПОСЛЕ БУЛЛИТОВ,Сан-Хосе,Нью-Джерси,3,4,32,2,41,4
0,США: НХЛ,17.01.2023 00:00,ЗАВЕРШЕН,Сиэтл,Тампа-Бэй,1,4,23,2,31,3
0,США: НХЛ,17.01.2023 02:00,ЗАВЕРШЕН,Вегас,Даллас,0,4,27,2,36,2
...,...,...,...,...,...,...,...,...,...,...,...
0,ШВЕЙЦАРИЯ: ШВЕЙЦАРСКАЯ ЛИГА - ТУР 39,17.01.2023 21:45,,Базель,Тичино Рокетс,-,-,_,_,_,_
0,ШВЕЙЦАРИЯ: ШВЕЙЦАРСКАЯ ЛИГА - ТУР 39,17.01.2023 21:45,,Шо-де-Фондз,Ольтен,-,-,_,_,_,_
0,ШВЕЙЦАРИЯ: ШВЕЙЦАРСКАЯ ЛИГА - ТУР 39,17.01.2023 21:46,,Лангенталь,Еллоу Винтертур,-,-,_,_,_,_
0,ШВЕЙЦАРИЯ: ШВЕЙЦАРСКАЯ ЛИГА - ТУР 39,17.01.2023 22:00,,Тургау,Висп,-,-,_,_,_,_
